In [10]:
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras import layers
import os

from process_ntuples import *

In [11]:
dir_path = "/global/cfs/cdirs/atlas/gottino/Charm/Analysis/run_batch_gup/ntuples"

files = os.listdir(dir_path)
files

['CharmAnalysis.mc.ForcedDecay.Dplus.tree.root',
 'CharmAnalysis.mc.ForcedDecay.LabmdaC.tree.root']

In [12]:
TTree = process_ntuple(dir_path, files[0])
TTree.name

'CharmAnalysis'

In [109]:
DMesons_df = show(0, TTree, ["DMesons_daughterInfo__pt", "DMesons_daughterInfo__eta", "DMesons_daughterInfo__phi", "DMesons_daughterInfo__pdgId"], 1000)
DMesons_df = remove_empty(DMesons_df)
DMesons_df

,DMesons_daughterInfo__pt,DMesons_daughterInfo__eta,DMesons_daughterInfo__phi,DMesons_daughterInfo__pdgId
0,"((1118.3367, 3805.8997, 971.6052), (3805.8997,...","((1.0579262, 0.8759657, 1.0816675), (0.8759657...","((1.2175969, 1.3583351, 0.71336174), (1.358335...","((-321, 211, 211), (2212, -321, 211))"
1,"((6094.893, 1307.462, 2733.4648))","((0.15998553, 0.011107716, -0.0614343))","((2.6128821, 2.7195172, 2.9228168))","((-321, 211, 211))"
2,"((6457.058, 5970.0933, 6637.575), (6605.627, 6...","((-1.608788, -1.6307458, -1.7315177), (-1.7326...","((-0.371787, -0.5164238, -0.4971406), (-0.4966...","((-321, 211, 211), (2212, -321, 211))"
3,"((3794.4978, 3308.3105, 3065.4814), (3308.3105...","((-0.073717244, 0.22605924, -0.017923404), (0....","((0.753864, 0.6827104, 0.920779), (0.6827104, ...","((-321, 211, 211), (2212, -321, 211))"
4,"((3781.5564, 7396.4673, 1412.3259), (7396.4673...","((-0.9284433, -1.182188, -1.1012387), (-1.1821...","((0.8518817, 0.7419038, 0.77916163), (0.741903...","((-321, 211, 211), (2212, -321, 211))"
...,...,...,...,...
556,"((2248.0342, 1560.9309, 1777.5198), (1776.7628...","((-0.27776414, -0.030297821, 0.18143338), (0.1...","((2.6425247, 2.1114702, 2.3337889), (2.3332653...","((-321, 211, 211), (2212, -321, 211))"
557,"((2843.811, 12120.83, 15722.865), (12120.83, 2...","((1.6837795, 1.7058395, 1.7437785), (1.7058395...","((-0.5526616, -0.51433957, -0.4968482), (-0.51...","((-321, 211, 211), (2212, -321, 211), (2212, -..."
558,"((5232.3657, 5543.935, 850.05676), (5543.935, ...","((-0.24407308, -0.15124202, -0.6015392), (-0.1...","((0.29564288, 0.23316427, -0.011730594), (0.23...","((-321, 211, 211), (2212, -321, 211))"
559,"((5233.4795, 1644.8683, 2888.8506))","((-0.20566566, 0.1395557, -0.22413868))","((-1.8748832, -1.9805108, -1.8100455))","((2212, -321, 211))"


In [126]:
tuple_list = DMesons_df.to_records(index=False).tolist()

for i in range(len(tuple_list)):
    if i == 5:
        break
    for j in range(len(tuple_list[i])):
        if j == 3:
            print(tuple_list[i][j])

[[-321, 211, 211], [2212, -321, 211]]
[[-321, 211, 211]]
[[-321, 211, 211], [2212, -321, 211]]
[[-321, 211, 211], [2212, -321, 211]]
[[-321, 211, 211], [2212, -321, 211]]


In [176]:
DPlus = [-321, 211, 211]
LambdaC = [2212, -321, 211]

count = 0

remove_lambda = []

for index, row in DMesons_df.iterrows():
    if count == 500:
        break
    count+=1
    
    pts = row['DMesons_daughterInfo__pt']
    etas = row['DMesons_daughterInfo__eta']
    phis = row['DMesons_daughterInfo__phi']
    pdgids = row["DMesons_daughterInfo__pdgId"]
    
    #loop through until the first lambda encountered
    for i in range(len(pdgids) - 1):
        #if there are multiple lambda baryons
        if len(pdgids) > 2:
            #print(np.shape(pdgids))
            #stop at the first lambda and keep the higher proton pt valued lambda
            if list(pdgids[i]) == LambdaC:
                l1_pt, l2_pt = pts[i][0], pts[i + 1][0]
                if (l1_pt > l2_pt):
                    #Lambda 1's proton has higher pt
                    remove_lambda.append((index, i + 1))
                elif (l1_pt < l2_pt):
                    #Lambda 2's proton has higher pt!
                    remove_lambda.append((index, i))
                else:
                    raise Exception("Error: Are the proton pts equal?")

#takes tuples of shape (2,) / (index, column)
def remove_particles_from_df(tuple_list):
    for tup in tuple_list:
        index, column = tup
        print(index, column)

remove_particles_from_df(remove_lambda[:10])

32 2
46 1
48 1
66 2
70 1
72 1
77 2
84 1
88 2
94 1


In [81]:
model = Sequential()
model.add(tf.keras.Input(shape = (23)))
model.add(Dense(6, activation = "relu"))
model.add(Dense(1, activation = "relu"))

loss=tf.keras.losses.MeanSquaredError()
model.compile(loss= loss, optimizer= tf.keras.optimizers.Adam(learning_rate = 0.01))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 6)                 144       
                                                                 
 dense_1 (Dense)             (None, 1)                 7         
                                                                 
Total params: 151
Trainable params: 151
Non-trainable params: 0
_________________________________________________________________


2023-03-30 21:13:34.071910: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-03-30 21:13:34.072000: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (cori13): /proc/driver/nvidia/version does not exist
2023-03-30 21:13:34.078976: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
